In [ ]:
import os
import spacy
import lt_core_news_lg
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import pickle

In [ ]:
nlp = lt_core_news_lg.load()

In [ ]:
text = ""

files = [i for i in os.listdir("./Texts/1_batch") if i.endswith(".txt")]
for f in files :
    with open("./Texts/1_batch/"+ f ) as file :
        text +="\n\n" + file.read()


In [ ]:
doc = nlp( text )

In [ ]:

pos = ["ADJ" , "ADV"]

words = [ ]
vectors = []

for tok in doc :
    if tok.pos_ in pos and tok.has_vector == True :
        word = tok.lemma_.lower()
        if word not in words :
            words.append( word )
            vectors.append( tok.vector )

print("Sentiment words found: ")
print(len(words))


In [ ]:
df = pd.DataFrame()
df["words"] = words
df["vectors"] = vectors

df = shuffle( df , random_state=1001)

In [ ]:
df = df.tail(2000)

In [ ]:
features = np.array( list(df["vectors"]))
features.shape

In [ ]:
with open("SentModel_1stGen.pkl" , "rb") as file :
    model = pickle.load(file)

In [ ]:
preds = model.predict(features)

In [ ]:
words = list(df["words"])
vectors = list(df["vectors"])
pred_labels = []
sel_words=[]
sel_vectors = []

for index, item in enumerate( preds ):
    if item != 0 :
        pred_labels.append( item )
        sel_words.append(words[index])
        sel_vectors.append(vectors[index])
print(len(sel_words))

In [ ]:
label_correct = []

values = ["1" , "0" , "-1" , "break" ]

for index, item in enumerate( sel_words ):

    print(index)
    lab = input(f"{item} is {pred_labels[index]}")

    while lab not in values :
        lab = input(f"{item} is {pred_labels[index]}")
    if lab == "break":
        break
    else:
        label_correct.append( int(lab) )    

In [ ]:

print("Labels: ", len(label_correct))
print("POS: " , label_correct.count(1))
print("NEG: ", label_correct.count(-1))



In [ ]:
df2 = pd.DataFrame()

df2["words"] = sel_words
df2["vectors"] = sel_vectors
df2["labels"] = label_correct

print(df2.shape)
print(df2.head())

df2.to_pickle( "LabelledData_2.pkl" )
